In [6]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout
import sklearn
from sklearn.model_selection import train_test_split
import os
from os import listdir
from zipfile import ZipFile
import pickle
import cv2
import gdown
from tensorflow.keras.utils import img_to_array
import numpy as np

In [7]:
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None:
            img = cv2.resize(image, (32,32))
            return img_to_array(img)
        else:
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None

In [18]:
def load_dataset(dir):
    ImgList, label_list = [], []
    try:
            print("Load Dataset -->")
            root_dir = listdir(dir)
            print("Root Directory : ", root_dir)
            for shape in root_dir:
                if shape == ".DS_Store":
                    root_dir.remove(dir)
            
            for shape in root_dir:
                faceShape_image_list = listdir(f"{dir}/{shape}")
                
                for faceShape in faceShape_image_list:
                    
                    if faceShape == ".DS_Store":
                        faceShape_image_lsit.remove(faceShape)
                        
                for faceShape_image in faceShape_image_list[:7000]:
                    
                    img_dir = f"{root_dir}/{shape}/{faceShape_image}"
                    
                    if img_dir.endswith(".jpg") == True or img_dir.endswith(".JPG") == True or img_dir.endswith(".png") == True or img_dir.endswith(".PNG") == True:
                        
                        converted_image = convert_image_to_array(img_dir)
                        ImgList.append(converted_image)
                        label_list.append(shape)
            print("Loading completed")
            return ImgList, label_list
    except Exception as e:
        print(f"Error : {e}")
        

In [27]:
images, labels = load_dataset("faceshape-master/published_dataset")

Load Dataset -->
Root Directory :  ['heart', 'oblong', 'oval', 'round', 'square']
Loading completed


In [24]:
print(len(images))

500
